<a href="https://colab.research.google.com/github/vnsdare/Jogo_Algoritmo/blob/main/Doutorado_algoritmo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

------
#<font size=6 color=grey> Algoritmo 1: U-Net enxuta
--------------------------------------

In [ ]:
# Step 1: Mount Google Drive and Install Libraries
from google.colab import drive
drive.mount('/content/drive')

!pip install -q nibabel tensorflow scikit-image

print("Libraries installed and Drive mounted successfully.")

# Step 2: Import Libraries and Define Configuration
import os
import glob
import random
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from scipy.ndimage import zoom, label, rotate
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Configuration (com as alterações para economizar RAM)
class Config:
    # Path Configuration
    BASE_PATH = "/content/drive/MyDrive/Python Files/ISLES-2022/ISLES-2022"
    DERIVATIVES_PATH = os.path.join(BASE_PATH, "derivatives")

    # Data & Model Parameters
    IMG_SIZE = (96, 96, 96)
    VOXEL_SPACING = (1.0, 1.0, 1.0)
    N_CHANNELS = 3
    N_MODELS = 3

    # Training Hyperparameters
    BATCH_SIZE = 1
    LEARNING_RATE = 1e-4
    EPOCHS = 3

CONFIG = Config()

print("Configuration updated to save RAM.")


# Step 3: Data Loading and Preprocessing Functions
def get_file_paths():
    patient_folders = sorted(glob.glob(os.path.join(CONFIG.BASE_PATH, "sub-strokecase*")))
    image_paths, mask_paths = [], []
    for patient_folder in patient_folders:
        patient_id = os.path.basename(patient_folder)
        try:
            dwi_path = glob.glob(os.path.join(patient_folder, "ses-0001/dwi/*_dwi.nii.gz"))[0]
            adc_path = glob.glob(os.path.join(patient_folder, "ses-0001/dwi/*_adc.nii.gz"))[0]
            flair_path = glob.glob(os.path.join(patient_folder, "ses-0001/anat/*_FLAIR.nii.gz"))[0]
            mask_path = glob.glob(os.path.join(CONFIG.DERIVATIVES_PATH, f"{patient_id}/ses-0001/*_msk.nii.gz"))[0]
            image_paths.append((dwi_path, adc_path, flair_path))
            mask_paths.append(mask_path)
        except IndexError:
            print(f"Skipping {patient_id} due to missing files.")
    return image_paths, mask_paths

def resample_volume(data, header, new_spacing=CONFIG.VOXEL_SPACING):
    original_spacing = header.get_zooms()
    resize_factor = [orig / new for orig, new in zip(original_spacing, new_spacing)]
    return zoom(data, resize_factor, order=1, mode='nearest')

def preprocess_volume(volume_data):
    mean, std = np.mean(volume_data), np.std(volume_data)
    return (volume_data - mean) / std if std > 0 else volume_data

def load_case(image_paths, mask_path):
    dwi_path, adc_path, flair_path = image_paths
    modalities = []
    dwi_nii = nib.load(dwi_path)
    dwi_data = resample_volume(dwi_nii.get_fdata(), dwi_nii.header)
    dwi_data = preprocess_volume(dwi_data)
    modalities.append(dwi_data)
    reference_shape = dwi_data.shape
    for path in [adc_path, flair_path]:
        img_nii = nib.load(path)
        img_data = resample_volume(img_nii.get_fdata(), img_nii.header)
        img_data = preprocess_volume(img_data)
        if img_data.shape != reference_shape:
            img_data = resize(img_data, reference_shape, order=1, preserve_range=True, anti_aliasing=True)
        modalities.append(img_data)
    stacked_images = np.stack(modalities, axis=-1)
    mask_nii = nib.load(mask_path)
    mask_data = resample_volume(mask_nii.get_fdata(), mask_nii.header)
    if mask_data.shape != reference_shape:
        mask_data = resize(mask_data, reference_shape, order=0, preserve_range=True, anti_aliasing=False)
    mask_data = (mask_data > 0).astype(np.float32)
    return stacked_images, np.expand_dims(mask_data, axis=-1)

print("Data helper functions defined.")


# Step 4: U-Net Model and 3D-Aware Augmentation
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)
    c1 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling3D((2, 2, 2))(c1)
    c2 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling3D((2, 2, 2))(c2)
    c3 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)
    u2 = layers.Conv3DTranspose(64, (2, 2, 2), strides=(2, 2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c2])
    c4 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c4)
    u1 = layers.Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(c4)
    u1 = layers.concatenate([u1, c1])
    c5 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(u1)
    c5 = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c5)
    outputs = layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(c5)
    return models.Model(inputs=[inputs], outputs=[outputs])

def augment_3d(image, mask):
    if random.random() > 0.5:
        image, mask = np.flip(image, axis=2), np.flip(mask, axis=2)
    if random.random() > 0.5:
        angle = random.uniform(-15, 15)
        image = rotate(image, angle, axes=(1, 2), reshape=False, order=1, mode='constant', cval=0)
        mask = rotate(mask, angle, axes=(1, 2), reshape=False, order=0, mode='constant', cval=0)
    if random.random() > 0.3:
        image += np.random.normal(0, 0.1, image.shape)
    return image, mask

print("U-Net and 3D Augmentation functions defined.")


# Step 5: Loss, Metrics, and Data Generator
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f, y_pred_f = K.flatten(y_true), K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def combined_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)

def crop_or_pad_volume_robust(volume, target_shape):
    d, h, w = volume.shape[:3]
    td, th, tw = target_shape
    d_start = (d - td) // 2 if d > td else 0
    h_start = (h - th) // 2 if h > th else 0
    w_start = (w - tw) // 2 if w > tw else 0
    cropped_volume = volume[d_start:d_start + td, h_start:h_start + th, w_start:w_start + tw, :]
    cd, ch, cw = cropped_volume.shape[:3]
    d_pad_before, d_pad_after = (td - cd) // 2, td - cd - ((td - cd) // 2)
    h_pad_before, h_pad_after = (th - ch) // 2, th - ch - ((th - ch) // 2)
    w_pad_before, w_pad_after = (tw - cw) // 2, tw - cw - ((tw - cw) // 2)
    padding_config = [(d_pad_before, d_pad_after), (h_pad_before, h_pad_after), (w_pad_before, w_pad_after), (0, 0)]
    final_volume = np.pad(cropped_volume, pad_width=padding_config, mode='constant')
    assert final_volume.shape[:3] == target_shape
    return final_volume

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths, batch_size, augment=False):
        self.image_paths, self.mask_paths = image_paths, mask_paths
        self.batch_size, self.augment = batch_size, augment
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_images, batch_masks = [], []
        for i in indices:
            img, mask = load_case(self.image_paths[i], self.mask_paths[i])
            if self.augment:
                img, mask = augment_3d(img, mask)
            img = crop_or_pad_volume_robust(img, CONFIG.IMG_SIZE)
            mask = crop_or_pad_volume_robust(mask, CONFIG.IMG_SIZE)
            batch_images.append(img)
            batch_masks.append(mask)
        return np.array(batch_images), np.array(batch_masks)

    def on_epoch_end(self):
        self.indices = np.arange(len(self.image_paths))
        np.random.shuffle(self.indices)

print("Loss, metrics, and robust data generator are ready.")


# Step 6: Training the Ensemble
print("Loading file paths...")
all_image_paths, all_mask_paths = get_file_paths()
train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(
    all_image_paths, all_mask_paths, test_size=0.2, random_state=42
)
train_generator = DataGenerator(train_img_paths, train_mask_paths, CONFIG.BATCH_SIZE, augment=True)
val_generator = DataGenerator(val_img_paths, val_mask_paths, CONFIG.BATCH_SIZE, augment=False)
print(f"Data prepared: {len(train_img_paths)} training, {len(val_img_paths)} validation cases.")

input_shape = CONFIG.IMG_SIZE + (CONFIG.N_CHANNELS,)
for i in range(CONFIG.N_MODELS):
    print(f"\n--- Training Model {i+1}/{CONFIG.N_MODELS} ---")
    model = build_unet(input_shape)
    model.compile(optimizer=optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE),
                  loss=combined_loss,
                  metrics=[dice_coefficient])
    model_checkpoint_path = f"/content/drive/MyDrive/isles22_unet_model_{i}.h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        model_checkpoint_path, monitor='val_dice_coefficient', save_best_only=True, mode='max'
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_dice_coefficient', patience=10, mode='max', restore_best_weights=True
    )

    # CORREÇÃO: O argumento 'workers' foi removido da chamada .fit()
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=CONFIG.EPOCHS,
        callbacks=[checkpoint, early_stopping]
    )
    print(f"Finished training Model {i+1}. Best weights saved to {model_checkpoint_path}")
print("\nEnsemble training complete.")


# Step 7: Evaluation
def lesion_wise_f1(y_true, y_pred):
    true_labels, num_true = label(y_true)
    pred_labels, num_pred = label(y_pred)
    if num_true == 0 and num_pred == 0: return 1.0
    if num_true == 0 or num_pred == 0: return 0.0
    true_positives = 0
    for i in range(1, num_true + 1):
        if np.sum(y_pred[true_labels == i]) > 0:
            true_positives += 1
    precision = true_positives / num_pred if num_pred > 0 else 0
    recall = true_positives / num_true if num_true > 0 else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def evaluate_ensemble(model_paths, val_generator):
    input_shape = CONFIG.IMG_SIZE + (CONFIG.N_CHANNELS,)
    ensemble_models = []
    for path in model_paths:
        model = build_unet(input_shape)
        model.load_weights(path)
        ensemble_models.append(model)

    all_dice_scores, all_f1_scores = [], []
    print("\n--- Evaluating Ensemble Performance ---")
    for i in tqdm(range(len(val_generator)), desc="Evaluating Validation Set"):
        X_val, y_val = val_generator[i]
        for j in range(X_val.shape[0]):
            image, mask_true = np.expand_dims(X_val[j], axis=0), y_val[j]
            predictions = [model.predict(image, verbose=0)[0] for model in ensemble_models]
            ensembled_pred_mask = (np.mean(predictions, axis=0) > 0.5).astype(np.uint8)
            dice_score = dice_coefficient(mask_true.astype(np.float32), ensembled_pred_mask.astype(np.float32)).numpy()
            f1_score = lesion_wise_f1(mask_true.squeeze(), ensembled_pred_mask.squeeze())
            all_dice_scores.append(dice_score)
            all_f1_scores.append(f1_score)

    avg_dice, avg_f1 = np.mean(all_dice_scores), np.mean(all_f1_scores)
    print(f"\n--- Ensemble Evaluation Results ---")
    print(f"Average Voxel-wise Dice Coefficient: {avg_dice:.4f}")
    print(f"Average Lesion-wise F1 Score: {avg_f1:.4f}")

# --- Run Evaluation ---
model_paths = [f"/content/drive/MyDrive/isles22_unet_model_{i}.h5" for i in range(CONFIG.N_MODELS)]
evaluate_ensemble(model_paths, val_generator)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Libraries installed and Drive mounted successfully.
Configuration updated to save RAM.
Data helper functions defined.
U-Net and 3D Augmentation functions defined.
Loss, metrics, and robust data generator are ready.
Loading file paths...
Data prepared: 200 training, 50 validation cases.

--- Training Model 1/3 ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_18']
Received: inputs=Tensor(shape=(None, 96, 96, 96, 3))
  warnings.warn(msg)


200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - dice_coefficient: 0.0449 - loss: 1.1801

...

200/200 ━━━━━━━━━━━━━━━━━━━━ 1208s 6s/step - dice_coefficient: 0.2661 - loss: 0.8462 - val_dice_coefficient: 0.2697 - val_loss: 0.8212
Epoch 3/3
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - dice_coefficient: 0.3736 - loss: 0.7280

200/200 ━━━━━━━━━━━━━━━━━━━━ 1210s 6s/step - dice_coefficient: 0.3735 - loss: 0.7281 - val_dice_coefficient: 0.2846 - val_loss: 0.7874
Finished training Model 3. Best weights saved to /content/drive/MyDrive/isles22_unet_model_2.h5

Ensemble training complete.

--- Evaluating Ensemble Performance ---


Evaluating Validation Set:   0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_72']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_90']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_108']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)



--- Ensemble Evaluation Results ---
Average Voxel-wise Dice Coefficient: 0.3594
Average Lesion-wise F1 Score: 0.2618


------
#<font size=6 color=grey> Algoritmo 2: ResNet + U-Net
--------------------------------------


In [ ]:
# Step 1: Mount Google Drive and Install Libraries

from google.colab import drive
drive.mount('/content/drive')

# Install required libraries quietly
!pip install -q nibabel tensorflow scikit-image

print("Libraries installed and Drive mounted successfully.")

# Step 2: Import Libraries and Define Configuration
# -----------------------------------------------------------------
# Here, we import all necessary modules and define a configuration class
# to manage file paths and model hyperparameters centrally. This makes
# the script easier to read and modify.

import os
import glob
import random
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from scipy.ndimage import zoom, label, rotate
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Configuration (lower computational cost and RAM usage).
class Config:
    # --- Path Configuration ---
    BASE_PATH = "/content/drive/MyDrive/Python Files/ISLES-2022/ISLES-2022"
    DERIVATIVES_PATH = os.path.join(BASE_PATH, "derivatives")

    # --- Data & Model Parameters ---
    IMG_SIZE = (96, 96, 96) # Using smaller dimensions to save memory
    VOXEL_SPACING = (2.0, 2.0, 2.0) # Resampling to a consistent, lower resolution
    N_CHANNELS = 3 # DWI, ADC, FLAIR
    N_MODELS = 3 # Number of models in the ensemble

    # --- Training Hyperparameters ---
    BATCH_SIZE = 1 # Use a batch size of 1 to minimize RAM usage
    LEARNING_RATE = 1e-4
    EPOCHS = 25 # Increased epochs for better convergence

CONFIG = Config()
print("Configuration loaded. Image size set to:", CONFIG.IMG_SIZE)


# Step 3: Data Loading and Preprocessing Functions
def get_file_paths():
    """
    Scans the dataset directory and returns sorted lists of image and mask file paths.
    This function is robust to the BIDS-like structure of the ISLES 2022 dataset.
    """
    patient_folders = sorted(glob.glob(os.path.join(CONFIG.BASE_PATH, "sub-strokecase*")))
    image_paths, mask_paths = [], []

    print(f"Found {len(patient_folders)} patient folders.")
    for patient_folder in patient_folders:
        patient_id = os.path.basename(patient_folder)
        session_folder = glob.glob(os.path.join(patient_folder, "ses-*"))

        if not session_folder:
            print(f"Warning: No session folder found for {patient_id}. Skipping.")
            continue
        session_folder = session_folder[0] # Assume one session per patient
        session_id = os.path.basename(session_folder)

        try:
            # Construct file paths based on the ISLES 2022 naming convention
            dwi_path = glob.glob(os.path.join(session_folder, "dwi", f"{patient_id}_{session_id}_dwi.nii.gz"))[0]
            adc_path = glob.glob(os.path.join(session_folder, "dwi", f"{patient_id}_{session_id}_adc.nii.gz"))[0]
            flair_path = glob.glob(os.path.join(session_folder, "anat", f"{patient_id}_{session_id}_FLAIR.nii.gz"))[0]
            mask_path = glob.glob(os.path.join(CONFIG.DERIVATIVES_PATH, patient_id, session_id, f"{patient_id}_{session_id}_msk.nii.gz"))[0]

            image_paths.append((dwi_path, adc_path, flair_path))
            mask_paths.append(mask_path)
        except IndexError:
            print(f"Warning: Skipping {patient_id} due to missing image or mask files.")

    return image_paths, mask_paths

def resample_volume(data, header, new_spacing=CONFIG.VOXEL_SPACING):
    """Resamples a 3D volume to a new voxel spacing."""
    original_spacing = header.get_zooms()[:3]
    resize_factor = [orig / new for orig, new in zip(original_spacing, new_spacing)]
    return zoom(data, resize_factor, order=1, mode='nearest')

def preprocess_volume(volume_data):
    """Normalizes the volume using Z-score normalization."""
    mean, std = np.mean(volume_data), np.std(volume_data)
    return (volume_data - mean) / std if std > 0 else volume_data

def load_case(image_paths, mask_path):
    """
    Loads, preprocesses, and stacks the three MRI modalities (DWI, ADC, FLAIR)
    and their corresponding lesion mask for a single case.
    """
    dwi_path, adc_path, flair_path = image_paths
    modalities = []

    # Load and process reference modality (DWI)
    dwi_nii = nib.load(dwi_path)
    dwi_data = resample_volume(dwi_nii.get_fdata(), dwi_nii.header)
    dwi_data = preprocess_volume(dwi_data)
    modalities.append(dwi_data)
    reference_shape = dwi_data.shape

    # Load and process other modalities, aligning to reference shape
    for path in [adc_path, flair_path]:
        img_nii = nib.load(path)
        img_data = resample_volume(img_nii.get_fdata(), img_nii.header)
        img_data = preprocess_volume(img_data)
        if img_data.shape != reference_shape:
            img_data = resize(img_data, reference_shape, order=1, preserve_range=True, anti_aliasing=False)
        modalities.append(img_data)

    stacked_images = np.stack(modalities, axis=-1)

    # Load and process mask, aligning to reference shape
    mask_nii = nib.load(mask_path)
    mask_data = resample_volume(mask_nii.get_fdata(), mask_nii.header)
    if mask_data.shape != reference_shape:
        mask_data = resize(mask_data, reference_shape, order=0, preserve_range=True, anti_aliasing=False)

    mask_data = (mask_data > 0).astype(np.float32)
    return stacked_images, np.expand_dims(mask_data, axis=-1)

print("Data helper functions defined.")


# Step 4: ResU-Net Model and 3D-Aware Augmentation

def resnet_block(input_tensor, num_filters):
    """A residual block with two 3x3 convolutions and a skip connection."""
    x = layers.Conv3D(num_filters, (3, 3, 3), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(num_filters, (3, 3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection to match dimensions if necessary
    shortcut = layers.Conv3D(num_filters, (1, 1, 1), padding='same')(input_tensor)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_resunet(input_shape):
    """Builds the ResU-Net model."""
    inputs = layers.Input(shape=input_shape)

    # Encoder Path
    c1 = resnet_block(inputs, 16)
    p1 = layers.MaxPooling3D((2, 2, 2))(c1)
    c2 = resnet_block(p1, 32)
    p2 = layers.MaxPooling3D((2, 2, 2))(c2)

    # Bottleneck
    c3 = resnet_block(p2, 64)

    # Decoder Path
    u2 = layers.Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c2])
    c4 = resnet_block(u2, 32)

    u1 = layers.Conv3DTranspose(16, (2, 2, 2), strides=(2, 2, 2), padding='same')(c4)
    u1 = layers.concatenate([u1, c1])
    c5 = resnet_block(u1, 16)

    outputs = layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(c5)

    return models.Model(inputs=[inputs], outputs=[outputs])


def augment_3d(image, mask):
    """Applies 3D data augmentation to an image and its mask."""
    # Random flip across the sagittal plane (left-right)
    if random.random() > 0.5:
        image = np.flip(image, axis=0)
        mask = np.flip(mask, axis=0)

    # Random rotation around the Z-axis (axial plane)
    if random.random() > 0.5:
        angle = random.uniform(-15, 15)
        image = rotate(image, angle, axes=(0, 1), reshape=False, order=1, mode='constant', cval=0)
        mask = rotate(mask, angle, axes=(0, 1), reshape=False, order=0, mode='constant', cval=0)

    # Add Gaussian noise
    if random.random() > 0.3:
        image += np.random.normal(0, 0.1, image.shape)

    return image, mask

print("ResU-Net model and 3D Augmentation functions defined.")


# Step 5: Loss, Metrics, and Data Generator

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """Calculates the Dice Similarity Coefficient (DSC)."""
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    """Calculates Dice loss from the Dice coefficient."""
    return 1 - dice_coefficient(y_true, y_pred)

def crop_or_pad_volume(volume, target_shape):
    """
    Crops or pads a 3D volume to a target shape.
    This ensures all inputs to the network are of a consistent size.
    """
    d, h, w = volume.shape[:3]
    td, th, tw = target_shape

    # Cropping
    d_start = max(0, (d - td) // 2)
    h_start = max(0, (h - th) // 2)
    w_start = max(0, (w - tw) // 2)
    cropped_volume = volume[d_start:d_start + td, h_start:h_start + th, w_start:w_start + tw, :]

    # Padding
    cd, ch, cw = cropped_volume.shape[:3]
    pad_dims = [
        ((td - cd) // 2, td - cd - ((td - cd) // 2)),
        ((th - ch) // 2, th - ch - ((th - ch) // 2)),
        ((tw - cw) // 2, tw - cw - ((tw - cw) // 2)),
        (0,0) # No padding for the channel dimension
    ]
    final_volume = np.pad(cropped_volume, pad_width=pad_dims, mode='constant')
    return final_volume

class DataGenerator(tf.keras.utils.Sequence):
    """Generates data for Keras."""
    def __init__(self, image_paths, mask_paths, batch_size, augment=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_images, batch_masks = [], []

        for i in indices:
            img, mask = load_case(self.image_paths[i], self.mask_paths[i])
            if self.augment:
                img, mask = augment_3d(img, mask)

            img = crop_or_pad_volume(img, CONFIG.IMG_SIZE)
            mask = crop_or_pad_volume(mask, CONFIG.IMG_SIZE)

            batch_images.append(img)
            batch_masks.append(mask)

        return np.array(batch_images), np.array(batch_masks)

    def on_epoch_end(self):
        self.indices = np.arange(len(self.image_paths))
        np.random.shuffle(self.indices)

print("Loss, metrics, and data generator are ready.")


# Step 6: Training the Ensemble of Models

print("Loading file paths...")
all_image_paths, all_mask_paths = get_file_paths()

if not all_image_paths:
    print("\nERROR: No data files were found. Please check the `BASE_PATH` in Step 2.")
    print("Expected path:", CONFIG.BASE_PATH)
else:
    # Split data into training and validation sets
    train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(
        all_image_paths, all_mask_paths, test_size=0.2, random_state=42
    )

    # Create data generators
    train_generator = DataGenerator(train_img_paths, train_mask_paths, CONFIG.BATCH_SIZE, augment=True)
    val_generator = DataGenerator(val_img_paths, val_mask_paths, CONFIG.BATCH_SIZE, augment=False)

    print(f"Data prepared: {len(train_img_paths)} training, {len(val_img_paths)} validation cases.")

    input_shape = CONFIG.IMG_SIZE + (CONFIG.N_CHANNELS,)

    for i in range(CONFIG.N_MODELS):
        print(f"\n--- Training Model {i+1}/{CONFIG.N_MODELS} ---")
        model = build_resunet(input_shape)
        model.compile(
            optimizer=optimizers.Adam(learning_rate=CONFIG.LEARNING_RATE),
            loss=dice_loss,
            metrics=[dice_coefficient, 'accuracy']
        )

        model_checkpoint_path = f"/content/drive/MyDrive/isles22_resunet_model_{i}.h5"
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            model_checkpoint_path, monitor='val_dice_coefficient', save_best_only=True, mode='max'
        )
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_dice_coefficient', patience=10, mode='max', restore_best_weights=True
        )

        history = model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=CONFIG.EPOCHS,
            callbacks=[checkpoint, early_stopping],
        )
            print(f"Finished training Model {i+1}. Best weights saved to {model_checkpoint_path}")

    print("\nEnsemble training complete.")

# Step 7: Evaluation of the Ensemble

def lesion_wise_f1(y_true, y_pred):
    """Calculates the lesion-wise F1 score."""
    true_labels, num_true = label(y_true > 0.5)
    pred_labels, num_pred = label(y_pred > 0.5)

    if num_true == 0 and num_pred == 0: return 1.0
    if num_true == 0 or num_pred == 0: return 0.0

    true_positives = 0
    for i in range(1, num_true + 1):
        if np.sum(y_pred[true_labels == i]) > 0:
            true_positives += 1

    precision = true_positives / num_pred if num_pred > 0 else 0
    recall = true_positives / num_true if num_true > 0 else 0

    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def evaluate_ensemble(model_paths, val_generator):
    """Evaluates the performance of the model ensemble."""
    input_shape = CONFIG.IMG_SIZE + (CONFIG.N_CHANNELS,)
    ensemble_models = []
    for path in model_paths:
        model = build_resunet(input_shape)
        model.load_weights(path)
        ensemble_models.append(model)

    all_dice_scores, all_f1_scores = [], []
    print("\n--- Evaluating Ensemble Performance ---")

    for i in tqdm(range(len(val_generator)), desc="Evaluating Validation Set"):
        X_val, y_val = val_generator[i]
        for j in range(X_val.shape[0]):
            image, mask_true = np.expand_dims(X_val[j], axis=0), y_val[j]

            # Get predictions from all models in the ensemble
            predictions = [model.predict(image, verbose=0)[0] for model in ensemble_models]

            # Average the predictions and apply a 0.5 threshold
            ensembled_pred_mask = (np.mean(predictions, axis=0) > 0.5).astype(np.uint8)

            dice_score = dice_coefficient(mask_true.astype(np.float32), ensembled_pred_mask.astype(np.float32)).numpy()
            f1_score = lesion_wise_f1(mask_true.squeeze(), ensembled_pred_mask.squeeze())

            all_dice_scores.append(dice_score)
            all_f1_scores.append(f1_score)

    avg_dice = np.mean(all_dice_scores)
    avg_f1 = np.mean(all_f1_scores)

    print(f"\n--- Ensemble Evaluation Results ---")
    print(f"Average Voxel-wise Dice Coefficient: {avg_dice:.4f}")
    print(f"Average Lesion-wise F1 Score: {avg_f1:.4f}")

# --- Run Evaluation ---
if 'val_generator' in locals():
    model_paths = [f"/content/drive/MyDrive/isles22_resunet_model_{i}.h5" for i in range(CONFIG.N_MODELS)]
    evaluate_ensemble(model_paths, val_generator)
else:
    print("\nSkipping evaluation because training data was not loaded.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Libraries installed and Drive mounted successfully.
Configuration loaded. Image size set to: (96, 96, 96)
Data helper functions defined.
ResU-Net model and 3D Augmentation functions defined.
Loss, metrics, and data generator are ready.
Loading file paths...
Found 250 patient folders.
Data prepared: 200 training, 50 validation cases.

--- Training Model 1/3 ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_53']
Received: inputs=Tensor(shape=(None, 96, 96, 96, 3))
  warnings.warn(msg)


200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8367 - dice_coefficient: 0.0223 - loss: 0.9777

...

200/200 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.9985 - dice_coefficient: 0.7146 - loss: 0.2854 - val_accuracy: 0.9988 - val_dice_coefficient: 0.5850 - val_loss: 0.4150
Epoch 24/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9984 - dice_coefficient: 0.7111 - loss: 0.2889

200/200 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.9984 - dice_coefficient: 0.7111 - loss: 0.2889 - val_accuracy: 0.9989 - val_dice_coefficient: 0.6000 - val_loss: 0.4000
Epoch 25/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.9987 - dice_coefficient: 0.7028 - loss: 0.2972 - val_accuracy: 0.9968 - val_dice_coefficient: 0.4516 - val_loss: 0.5484
Finished training Model 3. Best weights saved to /content/drive/MyDrive/isles22_resunet_model_2.h5

Ensemble training complete.

--- Evaluating Ensemble Performance ---


Evaluating Validation Set:   0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_212']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_265']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_318']
Received: inputs=Tensor(shape=(1, 96, 96, 96, 3))
  warnings.warn(msg)



--- Ensemble Evaluation Results ---
Average Voxel-wise Dice Coefficient: 0.6253
Average Lesion-wise F1 Score: 0.6364
